In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.optim import AdamW
pd.options.mode.chained_assignment = None
import warnings
import random

Matplotlib is building the font cache; this may take a moment.


In [3]:
import torch
import torch.nn as nn
from model import ManZoneTransformer

In [8]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

model = ManZoneTransformer(
    feature_len=5,    # num of input features (x, y, v_x, v_y, defense)
    model_dim=64,     # experimented with 96 & 128... seems best
    num_heads=2,      # 2 seems best (but may have overfit when tried 4... may be worth iterating)
    num_layers=4,
    dim_feedforward=64 * 4,
    dropout=0.1,      # 10% dropout to prevent overfitting... iterate as model becomes more complex (industry std is higher, i believe)
    output_dim=2      # man or zone classification
).to(device)

In [10]:
best_model_path = f"best_model_week3.pth"
model.load_state_dict(torch.load(
    best_model_path, 
    weights_only=True,
    map_location=torch.device('cpu') 
    ))
model.eval()

ManZoneTransformer(
  (feature_norm_layer): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (feature_embedding_layer): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (3): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (drop

In [30]:
import requests

API_URL = "https://nfl-f1ma.onrender.com"
input_text = "2022110700_126"
gameId, playId = input_text.split('_')
endpoint = f"{API_URL}/tracking/?gameId={gameId}&playId={playId}"
try:
    response = requests.get(endpoint)
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()  # Print the API response (if applicable)
except requests.exceptions.RequestException as e:
    print(f"Error sending data to API: {e}")
data = data['data']

In [31]:
import pandas as pd
import math 

los, direction = None, None
df = pd.DataFrame(data)
for idx, row in df.iterrows():
    if row['club'] == 'football':
        los = row['x']
        direction = row['playDirection']
        break
circles = []
for idx, row in df.iterrows():
    if row.club == 'football':
        continue
    circles.append({
        "pos": [round(row.x * 9 + 50, 2), round(row.y * 9 + 50, 2)],
        "label": row.displayName.split(' ')[-1],
        "color": (0, 0, 255) if (direction == 'left' and row['x'] < los) or (direction == 'right' and row['x'] > los) else (255, 0, 0),
        "dragging": False,
        "vector": [round((row.s * math.cos(float(row.dir) - 90)) * 9 + 50, 2), round((row.s * math.sin(float(row.dir) - 90)) * 9 + 50, 2)],
    })
pd.DataFrame(circles)

,pos,label,color,dragging,vector
0,"[593.24, 408.29]",Peters,"(0, 0, 255)",False,"[61.48, 51.74]"
1,"[587.93, 203.45]",Humphrey,"(0, 0, 255)",False,"[57.99, 52.17]"
2,"[659.12, 349.97]",Ramczyk,"(255, 0, 0)",False,"[49.94, 49.93]"
3,"[637.34, 389.48]",Bowser,"(0, 0, 255)",False,"[50.53, 50.08]"
4,"[699.8, 295.97]",Kamara,"(255, 0, 0)",False,"[49.92, 50.16]"
5,"[563.54, 367.88]",Clark,"(0, 0, 255)",False,"[47.91, 55.07]"
6,"[661.91, 391.64]",Hill,"(255, 0, 0)",False,"[49.91, 49.98]"
7,"[636.26, 298.49]",Smith,"(0, 0, 255)",False,"[51.28, 52.17]"
8,"[652.01, 321.26]",McCoy,"(255, 0, 0)",False,"[51.67, 53.09]"
9,"[654.53, 335.93]",Ruiz,"(255, 0, 0)",False,"[50.18, 50.0]"
